<img src="./Assets/1.1 AgWorkHeaderImage.png">

## Multi Agent Workflows

<img src="./Assets/4.1 Multi Agent.png">

---

#### Load All Keys

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Recall: AstraDB Vector Tool (Custom CrewAI Tools)

In [2]:
Token=os.environ["ASTRA_TOKEN"]
EndPoint=os.environ["ASTRA_ENDPOINT"]

In [3]:
from astrapy import DataAPIClient
from astrapy.data_types import DataAPIVector
from openai import OpenAI

def search_documents(query: str, top_k: int = 5) -> list[str]:
    # Set up OpenAI
    openai_client = OpenAI()
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query,
        encoding_format="float"
    ).data[0].embedding

    # Connect to Astra
    astra_client = DataAPIClient(Token)
    db = astra_client.get_database_by_api_endpoint(EndPoint)
    collection = db.get_collection("CollectionSmallEmbeddings")

    # Query Astra
    cursor = collection.find(
        sort={"$vector": DataAPIVector(query_embedding)},
        limit=top_k,
        include_similarity=True
    )

    # Return documents
    results = []
    for doc in cursor:
        text = doc.get("text") or str(doc)
        score = doc.get("$similarity", 0)
        results.append(f"[{score:.4f}] {text}")
    return results


In [4]:
from crewai.tools import BaseTool

class AstraSearchTool(BaseTool):
    name: str = "Vector Tool"
    description: str = "Search the vector store for information about employee leave policy of AKAIWorks"

    def _run(self, query: str) -> str:
        # This is where your vector search logic goes
        results = search_documents(query)
        return "\n".join(results)

In [5]:
astra_tool = AstraSearchTool()

---

In [6]:
from crewai import LLM

llm = LLM(
    model="gpt-4o-mini",
    temperature=0.8,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

## Agents

In [7]:
from crewai import Agent
from crewai_tools import SerperDevTool

In [8]:
task_planner = Agent(
    role="Task Planning Analyst",
    goal="Break down user questions into subtasks.",
    backstory="You understand complex goals and decompose them into logical steps.",
    llm=llm, verbose=True
)

In [9]:
query_refiner = Agent(
    role="Query Refinement Specialist",
    goal="Improve vague or broad queries for better retrieval.",
    backstory="You turn fuzzy queries into laser-focused search prompts.",
    llm=llm, verbose=True
)


In [10]:
tool_selector = Agent(
    role="Retrieval Strategy Advisor",
    goal="Choose which data source (Web, Vector, LLM) is best.",
    backstory="You understand which retrieval method fits which query.",
    llm=llm, verbose=True
)

In [12]:
iterative_retriever = Agent(
    role="Iterative Retrieval Specialist",
    goal="Retrieve complete, high-quality information through iterative reflection and refinement.",
    backstory=(
        "You're a methodical research expert. Instead of stopping at one search, you evaluate your "
        "initial results, refine your approach, and perform additional rounds of retrieval if needed. "
        "You're committed to ensuring no key detail is missed."
    ),
    tools=[SerperDevTool(), astra_tool], 
    llm=llm,
    verbose=True
)

In [13]:
context_builder = Agent(
    role="Context Curator",
    goal="Select top passages from retrieved results.",
    backstory="You’re an information architect focused on building accurate context windows.",
    llm=llm, verbose=True
)

In [14]:
summarizer = Agent(
    role="Content Summarizer",
    goal="Summarize large retrieved content when needed.",
    backstory="You condense dense material into clean, clear summaries.",
    llm=llm, verbose=True
)


In [15]:
generator = Agent(
    role="Final Answer Composer",
    goal="Generate a clean, helpful, source-aware answer.",
    backstory="You produce clear answers from compiled research.",
    llm=llm, verbose=True
)


In [16]:
critic = Agent(
    role="AI Response Critic",
    goal="Check final answer for factuality and clarity.",
    backstory="You verify correctness and offer revisions when needed.",
    llm=llm, verbose=True
)

## Tasks

In [17]:
from crewai import Task

task_planner_task = Task(
    description="Analyze the user question '{user_input}' and break it into subtasks.",
    expected_output=(
        "A step-by-step plan. Example:\n"
        "1. Refine the query\n2. Retrieve info\n3. Generate answer\n4. Critique result"
    ),
    agent=task_planner
)

In [19]:
query_refinement_task = Task(
    description="Refine this query: '{user_input}' so it’s optimized for search and retrieval.",
    expected_output="A clear, specific refined query.",
    agent=query_refiner
)

In [20]:
tool_strategy_task = Task(
    description="Given the refined query, choose whether to use [WEB], [VECTOR], or [LLM]. Justify your choice.",
    expected_output="List like [WEB, VECTOR] and a short rationale.",
    agent=tool_selector
)

In [22]:
iterative_retrieval_task = Task(
    description=(
        "Use the refined query to perform information retrieval. "
        "Search relevant sources — use Serper for [WEB], VectorTool for [VECTOR]. "
        "After retrieving, reflect on the completeness and quality of results. "
        "If needed, refine your query and search again. Do this iteratively (up to 3 rounds max). "
        "At the end, return the best information you've gathered from all iterations, summarized by source."
    ),
    expected_output=(
        "A structured summary:\nWEB: ...\nVECTOR: ...\nIterations used: N\n"
        "Content must be accurate and comprehensive."
    ),
    agent=iterative_retriever,
    tools=[SerperDevTool(), astra_tool]
)

In [23]:
context_builder_task = Task(
    description="From the retrieval results, pick the most relevant content to build a high-quality context.",
    expected_output="A structured, clean context window.",
    agent=context_builder
)

In [24]:
summarization_task = Task(
    description="If the context is too long, summarize it clearly without losing key facts.",
    expected_output="~200 word summary of the context content.",
    agent=summarizer
)

In [25]:
generation_task = Task(
    description="Write the final answer to '{user_input}' using the context. Mention sources used.",
    expected_output="Final answer with a [Sources used: ...] section.",
    agent=generator
)

In [26]:
critic_task = Task(
    description="Critique the answer for accuracy, clarity, and completeness. Suggest edits if needed.",
    expected_output="Evaluation: [Approved] or [Needs Revision] + possible revision.",
    agent=critic
)

## Crew

In [27]:
from crewai import Crew, Process

rag_sequential_crew = Crew(
    agents=[
        task_planner,
        query_refiner,
        tool_selector,
        iterative_retriever,
        context_builder,
        summarizer,
        generator,
        critic,
    ],
    tasks=[
        task_planner_task,
        query_refinement_task,
        tool_strategy_task,
        iterative_retrieval_task,
        context_builder_task,
        summarization_task,
        generation_task,
        critic_task,
    ],
    process=Process.sequential
)


In [28]:
user_input = input("🧠 Enter your question: ")
result = rag_sequential_crew.kickoff(inputs={"user_input": user_input})

# Agent: Task Planning Analyst
## Task: Analyze the user question 'How does the leave policy of AKAIWorks conform with the latest labour laws in India?' and break it into subtasks.


# Agent: Task Planning Analyst
## Final Answer: 
1. Refine the query: Understand the specific aspects of the leave policy of AKAIWorks that need to be analyzed in relation to the latest labour laws in India, such as types of leave (casual, sick, paid, unpaid), eligibility criteria, and any recent updates in labour regulations.

2. Retrieve info: Collect the current leave policy details of AKAIWorks, including all types of leave offered, application procedures, and any previous changes made. Additionally, gather information on the latest labour laws in India regarding employee leave entitlements, amendments, and compliance requirements.

3. Generate answer: Analyze the retrieved information to identify any discrepancies or alignments between AKAIWorks' leave policy and the latest Indian labour laws. Prepare

In [29]:
from IPython.display import Markdown, display
display(Markdown("### "+ result.raw))

### The leave policy of AKAIWorks is designed to align closely with the latest labour laws in India, providing various types of leave while ensuring compliance with statutory requirements. 

### AKAIWorks Employee Leave Policy Overview

1. **Types of Leave Offered**:
   - **Casual Leave**: Employees are entitled to casual leave for personal matters, as per company guidelines.
   - **Sick Leave**: Employees can take sick leave when ill, typically requiring medical documentation to support their absence.
   - **Paid Leave**: AKAIWorks provides paid leave, allowing employees to take time off without losing income.
   - **Unpaid Leave**: Unpaid leave may be granted in specific situations, subject to management approval.
   - **Maternity Leave**: The policy includes maternity leave in compliance with statutory mandates, ensuring employees receive their entitled benefits.

2. **Eligibility Criteria**:
   - The leave policy applies to all employee categories, including full-time, part-time, temporary, and contract workers.
   - Certain leaves, such as sick leave, may have conditions that need to be fulfilled (e.g., submission of a medical certificate).

3. **Application Procedures**:
   - Employees are required to submit formal leave requests through the established process, which includes notifying their direct supervisor and completing necessary paperwork.
   - The policy specifies procedures for managing leave balances and for employees who are rehired.

4. **Compliance with Indian Labour Laws**:
   - AKAIWorks emphasizes adherence to the latest Indian labour laws, stating that if any discrepancies arise between the company's policy and legal provisions, the latter will take precedence.
   - The policy is structured to align with recent amendments in labour regulations regarding employee leave entitlements.

### Latest Labour Laws in India Overview

1. **Leave Entitlements**:
   - The latest labour laws mandate employees' rights to various types of leave, including maternity, sick, and casual leave. Employers are obligated to provide a minimum number of leaves and comply with statutory requirements related to employee benefits (Source: [ICLG](https://iclg.com/practice-areas/employment-and-labour-laws-and-regulations/india), [Jibble](https://www.jibble.io/labor-laws/india-labour-laws)).

2. **Duties and Responsibilities**:
   - Employers must ensure their leave policies meet the minimum standards set by law, which has been updated under the new labour codes affecting employee benefits and social security entitlements (Source: [Labour Ministry](https://labour.gov.in/sites/default/files/labour_code_eng.pdf)).

3. **Compliance Requirements**:
   - A comprehensive compliance guide is available to help employers align their policies with legal standards, ensuring employees’ statutory rights concerning leave entitlements are upheld (Source: [Remote](https://remote.com/blog/employment-laws-india-compliance)).

### Comparison Analysis

- **Alignment**: AKAIWorks’ leave policy aligns well with Indian labour laws by offering a range of leave types and ensuring compliance with statutory rights. The provision for maternity leave and the inclusion of different employee categories reflect this commitment.
- **Deviations**: The policy explicitly states that any inconsistencies will defer to legal provisions, highlighting the company's priority on legal compliance.

In conclusion, AKAIWorks’ leave policy not only meets but also prioritizes compliance with current Indian labour laws, fostering a supportive work environment while ensuring that employees are aware of their rights and entitlements. 

[Sources used: ICLG, Jibble, Labour Ministry, Remote]

In [30]:
result.token_usage

UsageMetrics(total_tokens=28872, prompt_tokens=25387, cached_prompt_tokens=5760, completion_tokens=3485, successful_requests=11)

---